In [ ]:
import requests as req
import pandas as pd
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.support.select import Select
from selenium.webdriver.common.keys import Keys
import re
import time
import spacy
nlp = spacy.load("en_core_web_md")

In [ ]:
def get_ent(result_out,doc_txt):
    doc = nlp(doc_txt)
    ORG=[]
    Dates=[]
    Money=[]
    Other=[]
    for ent in doc.ents:
        if ent.label_.upper() in ['PERSON','ORG']:
            ORG.append(ent.text)
        elif ent.label_.upper() in ['TIME','DATE']:
            Dates.append(ent.text)
        elif ent.label_.upper() in ['MONEY']:
            Money.append(ent.text)
        else:
            Other.append(ent.text)

    add_to_dict(result_out,'ORG','|'.join(ORG))
    add_to_dict(result_out,'Money','|'.join(Money))
    add_to_dict(result_out,'Other','|'.join(Other))
    add_to_dict(result_out,'Dates','|'.join(Dates))

In [ ]:
result_out={}

In [ ]:
# doc = nlp(" Lupin contributes Rs 21 crores towards COVID-19 relief efforts EP News Bureau Apr 11, 2020 The contributions were made separately by Lupin’s employees, the Company and the promoter family")


In [ ]:
def add_to_dict(dictionary, key, value):
    if type(value)==list:
        for val in value:
            setKey(dictionary,key,val)
    else:
        if key not in dictionary:
            dictionary[key] = [value]
        elif type(dictionary[key]) == list:
            dictionary[key].append(value)
        else:
            dictionary[key] = [dictionary[key], value]

def get_filtered_article(article_list,keyword_list,result_out,page_no):
    for article in article_list:
        art_txt=article.text
        art_txt_clean=re.sub("[^A-Za-z0-9 ]", " ",art_txt)
        art_txt_clean=re.sub(" +", " ",art_txt_clean).lower()
        match_found=bool(re.search(keyword_list,art_txt_clean,re.IGNORECASE))
        art_url_title=article.find_element_by_tag_name('a')
        attr_url=''
        attr_url=art_url_title.get_attribute('href')
        # print(attr_url)
        # attr_title=art_url_title.text
        if match_found==True:
            match_words=re.findall(keyword_list,art_txt_clean,re.IGNORECASE)
            # add_to_dict(result_out,'Article_Title',attr_title)
            add_to_dict(result_out,'Article_Text',art_txt)
            add_to_dict(result_out,'Web_Page_No',page_no)
            add_to_dict(result_out,'Article_URL',attr_url)
            add_to_dict(result_out,'Matchd_words','|'.join(match_words))
            get_ent(result_out,art_txt)
        


In [ ]:
# os.chdir("C:\\Users\\DINES\\Desktop\\Development\\Amod_DataScience\\WebScrap\\TenderApp\\")
driver = webdriver.Chrome('chromedriver\\81\\chromedriver.exe')
driver.maximize_window()


## Keyword List

In [ ]:
keyword_list=(' contribut| aid| donate| fund')

### Extract Data for WHO

In [ ]:
page=driver.get('https://www.who.int/emergencies/diseases/novel-coronavirus-2019/donors-and-partners/funding')
ele_tbl=driver.find_element_by_tag_name('body')
tbl_HTML=ele_tbl.get_attribute("outerHTML")
soup=BeautifulSoup(tbl_HTML,'html.parser')
df_out=pd.read_html(tbl_HTML,header=0)
df_result=pd.DataFrame()
for df in df_out:
    df_result=df_result.append(df)
df_result.to_csv('Output/who.csv',index=False)

In [ ]:
df_result

### Extract Data For express Pharma

In [ ]:
result_out={}
# need to include data starting for this year
for page_index in range(1,14):
    driver.get('https://www.expresspharma.in/page/' + str(page_index) +'/?s=covid-19')
    article_list=driver.find_elements_by_tag_name('article')
    get_filtered_article(article_list,keyword_list,result_out,page_index)
df_out=pd.DataFrame.from_dict(result_out)
df_out.to_csv('output/express_pharma.csv')

### Extract Data for weforum

In [ ]:
result_out={}
pageNo=10
driver.get('https://www.weforum.org/search?query=covid-19')
for page_index in range(1,pageNo):
    time.sleep(2)
    driver.execute_script('window.scrollTo(0,document.body.scrollHeight);')
    next_ele=driver.find_elements_by_class_name('gsc-cursor-page')[page_index]
    next_ele.click()
    time.sleep(2)
    article_list=driver.find_elements_by_class_name('gsc-result')
    get_filtered_article(article_list,keyword_list,result_out,page_index)
df_out=pd.DataFrame.from_dict(result_out)
df_out.to_csv('output/weforum.csv')

### Extracting data for BBC

In [ ]:
article_list

In [ ]:
result_out={}
pageNo=3
for page_index in range(1,pageNo+1):
    driver.get('https://www.bbc.co.uk/search?q=covid-19+fund&page=' + str(page_index))
    article_list=driver.find_elements_by_class_name('css-14rwwjy-Promo')
    get_filtered_article(article_list,keyword_list,result_out,page_index)
df_out=pd.DataFrame.from_dict(result_out)
df_out.to_csv('output/bbc.csv')

### Extracting data for IndiaToday

In [ ]:
df_out=pd.DataFrame.from_dict(result_out)
df_out.to_csv('output/indiaToday.csv')

In [ ]:
result_out={}
pageNo=300
for page_index in range(1,pageNo+1):
    driver.get('https://www.indiatoday.in/topic/covid-19?page=' + str(page_index-1))
    article_list=driver.find_elements_by_class_name('itg-search-list')
    get_filtered_article(article_list,keyword_list,result_out,page_index)
df_out=pd.DataFrame.from_dict(result_out)
df_out.to_csv('output/indiaToday.csv')

In [ ]:
df_out

### Extracting Data for IMF

In [ ]:
result_out={}
# https://www.imf.org/en/Search#q=covid-19&sort=relevancy
# https://www.imf.org/en/Search#q=covid-19&first=10&sort=relevancy
pageNo=32
for page_index in range(0,pageNo):
    if page_index==0:
        driver.get('https://www.imf.org/en/Search#q=covid-19&sort=relevancy')
    else:
        driver.get('https://www.imf.org/en/Search#q=covid-19&first=' +str(page_index) +'0&sort=relevancy')
    time.sleep(3)
    article_list=driver.find_elements_by_class_name('coveo-result-item')
    get_filtered_article(article_list,keyword_list,result_out,page_index)
df_out=pd.DataFrame.from_dict(result_out)
df_out.to_csv('output/IMF.csv')

### Extracting data for ADB

In [ ]:
# https://www.adb.org/search?keywords=covid-19&page=1
result_out={}
pageNo=23
for page_index in range(1,pageNo+1):
    driver.get('https://www.adb.org/search?keywords=covid-19&page=' +str(page_index))
    # time.sleep(3)
    article_list=driver.find_elements_by_class_name('ola-snippet-inner')
    get_filtered_article(article_list,keyword_list,result_out,page_index)
df_out=pd.DataFrame.from_dict(result_out)
df_out.to_csv('output/ADB.csv')

### Extracting data for bloomberg

In [ ]:
result_out={}
dfOut=pd.DataFrame()
pageNo=3
driver.get('https://www.bloomberg.com/search?query=covid-19')
for page_index in range(1,pageNo+1):
    driver.execute_script('window.scrollTo(0,document.body.scrollHeight);')
    load_more=driver.find_element_by_class_name('button__e30d9cff')[1]
    time.sleep(2)
    load_more.click()
    # time.sleep(3)
article_list=driver.find_elements_by_class_name('storyItem__192ee8af')
get_filtered_article(article_list,keyword_list,result_out,page_index)
df_out=pd.DataFrame.from_dict(result_out)
df_out.to_csv('output/BloomBerg.csv')

### Extracting Data for New York Times

In [ ]:
df_out

In [ ]:
result_out={}
df_out=pd.DataFrame()
pageNo=20
driver.get('https://www.nytimes.com/search?query=covid-19')
next_btn=driver.find_element_by_class_name('css-vsuiox').find_element_by_tag_name('button')
for page_index in range(1,pageNo+1):
    driver.execute_script('window.scrollTo(0,document.body.scrollHeight);')
    time.sleep(2)
    next_btn.click()
time.sleep(3)
article_list=driver.find_elements_by_class_name('css-1l4w6pd')
get_filtered_article(article_list,keyword_list,result_out,page_index)
df_out=pd.DataFrame.from_dict(result_out)
df_out.to_csv('output/NYTimes.csv')

In [ ]:
y.click()
